In [1]:
# Matplotlib Inline Charts
%matplotlib inline

In [2]:
# Import Dependencies
from citipy import citipy
from datetime import datetime
import gmaps
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import requests
from scipy.stats import linregress
import time

# Import the API keys.
from config import weather_api_key
from config2 import g_key

In [3]:
# Load the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv(os.path.join("weather_data", "cities.csv"))
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Kruisfontein,ZA,2021-07-12 01:45:50,-34.0033,24.7314,47.03,55,0,2.59
1,1,Verkhovazhye,RU,2021-07-12 01:45:50,60.7472,42.0465,65.08,75,38,4.97
2,2,Richards Bay,ZA,2021-07-12 01:45:51,-28.7830,32.0377,63.19,76,0,13.78
3,3,Port Alfred,ZA,2021-07-12 01:45:51,-33.5906,26.8910,65.16,28,0,7.07
4,4,Jamestown,US,2021-07-12 01:43:22,42.0970,-79.2353,69.30,97,90,1.01


In [4]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [5]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [6]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
11,11,Butaritari,KI,2021-07-12 01:45:54,3.0707,172.7902,83.21,74,94,12.86
18,18,Hilo,US,2021-07-12 01:41:54,19.7297,-155.0900,78.80,68,1,5.75
19,19,Vaini,TO,2021-07-12 01:45:56,-21.2000,-175.2000,80.76,83,20,10.36
27,27,Meulaboh,ID,2021-07-12 01:45:59,4.1363,96.1285,77.45,77,97,3.89
29,29,Namatanai,PG,2021-07-12 01:45:59,-3.6667,152.4333,84.06,71,90,9.66
32,32,Westport,US,2021-07-12 01:46:00,41.1415,-73.3579,75.76,91,61,1.99
33,33,Tautira,PF,2021-07-12 01:46:01,-17.7333,-149.1500,82.42,80,15,17.63
42,42,Virginia Beach,US,2021-07-12 01:46:03,36.8529,-75.9780,82.47,80,75,6.91
43,43,Nakhon Phanom,TH,2021-07-12 01:46:04,17.3333,104.4167,80.40,71,9,4.00
47,47,Cabo San Lucas,MX,2021-07-12 01:46:05,22.8909,-109.9124,89.82,54,29,10.00


In [7]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
11,Butaritari,KI,83.21,3.0707,172.7902,
18,Hilo,US,78.80,19.7297,-155.0900,
19,Vaini,TO,80.76,-21.2000,-175.2000,
27,Meulaboh,ID,77.45,4.1363,96.1285,
29,Namatanai,PG,84.06,-3.6667,152.4333,
32,Westport,US,75.76,41.1415,-73.3579,
33,Tautira,PF,82.42,-17.7333,-149.1500,
42,Virginia Beach,US,82.47,36.8529,-75.9780,
43,Nakhon Phanom,TH,80.40,17.3333,104.4167,
47,Cabo San Lucas,MX,89.82,22.8909,-109.9124,


In [8]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [13]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        print("Added Hotel: " + hotels["results"][0]["name"])
    except (IndexError):
        print("Hotel not found... skipping.")

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Added Hotel: Isles Sunset Lodge
Added Hotel: Hilo Hawaiian Hotel
Added Hotel: Keleti Beach Resort
Added Hotel: Kost Pria
Added Hotel: Seagulls Inn Namatani
Added Hotel: The Westport Inn
Added Hotel: Pension Oaoa
Added Hotel: Hilton Virginia Beach Oceanfront
Added Hotel: ป่ายางรีสอร์ท
Added Hotel: Hotel Tesoro Los Cabos
Added Hotel: Casa Dora, Bubaque
Added Hotel: Bora-Bora Pearl Beach Resort & Spa
Added Hotel: Dhidhoo beach
Added Hotel: Hotel Mar Menor
Added Hotel: waigo splash hotel
Added Hotel: Paradise Inn
Added Hotel: The Hotel Petropavlovsky
Added Hotel: Hotel Narayan Avezika Comfort
Added Hotel: Cozumel Palace® All Inclusive Resort
Added Hotel: Hotel Durval
Hotel not found... skipping.
Added Hotel: Gostinitsa
Added Hotel: Hotel RIKMAN Continental
Added Hotel: Sheraton Kauai Coconut Beach Resort
Added Hotel: Scoop Guest House
Added Hotel: Villa Enata
Hotel not found... skipping.
Added Hotel: Grage Hotel Bengkulu
Added Hotel: Hotel Manglares
Added Hotel: Lovers Point Inn
Added Hote

In [14]:
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
11,Butaritari,KI,83.21,3.0707,172.7902,Isles Sunset Lodge
18,Hilo,US,78.80,19.7297,-155.0900,Hilo Hawaiian Hotel
19,Vaini,TO,80.76,-21.2000,-175.2000,Keleti Beach Resort
27,Meulaboh,ID,77.45,4.1363,96.1285,Kost Pria
29,Namatanai,PG,84.06,-3.6667,152.4333,Seagulls Inn Namatani
32,Westport,US,75.76,41.1415,-73.3579,The Westport Inn
33,Tautira,PF,82.42,-17.7333,-149.1500,Pension Oaoa
42,Virginia Beach,US,82.47,36.8529,-75.9780,Hilton Virginia Beach Oceanfront
43,Nakhon Phanom,TH,80.40,17.3333,104.4167,ป่ายางรีสอร์ท
47,Cabo San Lucas,MX,89.82,22.8909,-109.9124,Hotel Tesoro Los Cabos


In [17]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [18]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [19]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))